Importing Libraries

In [38]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical


Dowload

In [39]:
def fetch_and_prepare_data(ticker):
    df = yf.download(ticker, start="2022-01-01", end="2024-12-31")

    # Technical Indicators
    df["SMA_50"] = df["Close"].rolling(window=50).mean()

    ema_12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema_26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema_12 - ema_26
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Hist"] = df["MACD"] - df["MACD_Signal"]

    delta = df["Close"].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df["RSI_14"] = 100 - (100 / (1 + rs))

    df["Return_21D"] = df["Close"].pct_change(21)
    df["Support_20D"] = df["Close"].rolling(20).min()
    df["Resistance_20D"] = df["Close"].rolling(20).max()

    def classify_trend(row):
        try:
            if (float(row['RSI_14']) > 60) and (float(row['Close']) > float(row['SMA_50'])) and (float(row['MACD']) > float(row['MACD_Signal'])):
                return "Bullish"
            elif (float(row['RSI_14']) < 40) and (float(row['Close']) < float(row['SMA_50'])) and (float(row['MACD']) < float(row['MACD_Signal'])):
                return "Bearish"
            else:
                return "Sideways"
        except:
            return "Sideways"


    df.dropna(inplace=True)
    df["Trend_Label"] = df.apply(classify_trend, axis=1)
    return df


In [40]:
def prepare_lstm_data(df):
    features = [
        'RSI_14', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Hist',
        'Return_21D', 'Support_20D', 'Resistance_20D', 'Volume'
    ]
    label_map = {'Bullish': 0, 'Bearish': 1, 'Sideways': 2}
    df['Label'] = df['Trend_Label'].map(label_map)

    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(df[features])

    X_seq, y_seq = [], []
    sequence_length = 60

    for i in range(sequence_length, len(scaled_features)):
        X_seq.append(scaled_features[i-sequence_length:i])
        y_seq.append(df['Label'].iloc[i])

    X_seq = np.array(X_seq)
    y_seq_cat = to_categorical(np.array(y_seq), num_classes=3)

    return train_test_split(X_seq, y_seq_cat, test_size=0.2, random_state=42)


In [ ]:
def train_lstm_model(X_train, X_test, y_train, y_test):
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Suppressed output for clean logging
    model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return model, loss, accuracy


In [42]:
"""
ticker = "INFY.NS"
print(f"Fetching and preparing data for {ticker}...")
df = fetch_and_prepare_data(ticker)

X_train, X_test, y_train, y_test = prepare_lstm_data(df)

print("Training LSTM model...")
model, loss, accuracy = train_lstm_model(X_train, X_test, y_train, y_test)

print(f"\n✅ Model trained successfully.\n📉 Loss: {loss:.4f}\n🎯 Accuracy: {accuracy:.4f}")

"""

'\nticker = "INFY.NS"\nprint(f"Fetching and preparing data for {ticker}...")\ndf = fetch_and_prepare_data(ticker)\n\nX_train, X_test, y_train, y_test = prepare_lstm_data(df)\n\nprint("Training LSTM model...")\nmodel, loss, accuracy = train_lstm_model(X_train, X_test, y_train, y_test)\n\nprint(f"\n✅ Model trained successfully.\n📉 Loss: {loss:.4f}\n🎯 Accuracy: {accuracy:.4f}")\n\n'

In [43]:

"""
# Save model after training
model.save("lstm_stock_trend_model.h5")
print("✅ Model saved as 'lstm_stock_trend_model.h5'")
"""

'\n# Save model after training\nmodel.save("lstm_stock_trend_model.h5")\nprint("✅ Model saved as \'lstm_stock_trend_model.h5\'")\n'

In [44]:
def predict_latest_trend(model, df, features, scaler, sequence_length=60):
    # Prepare the most recent sequence
    latest_data = df[features].tail(sequence_length)
    scaled = scaler.transform(latest_data)
    sequence = np.expand_dims(scaled, axis=0)  # shape: (1, 60, features)

    prediction = model.predict(sequence)
    class_index = np.argmax(prediction)
    confidence = prediction[0][class_index]
    label_map = {0: "Bullish", 1: "Bearish", 2: "Sideways"}

    return label_map[class_index], confidence


In [45]:
NIFTY 50

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2301097310.py, line 1)

In [46]:
import os

# Create folder for models
os.makedirs("Models", exist_ok=True)

# List of NIFTY 50 tickers
nifty_50_tickers = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS", "HINDUNILVR.NS",
    "SBIN.NS", "BHARTIARTL.NS", "ITC.NS", "LT.NS", "ASIANPAINT.NS", "AXISBANK.NS",
    "BAJFINANCE.NS", "KOTAKBANK.NS", "MARUTI.NS", "HCLTECH.NS", "SUNPHARMA.NS",
    "WIPRO.NS", "NESTLEIND.NS", "ULTRACEMCO.NS", "TITAN.NS", "TECHM.NS", "TATAMOTORS.NS",
    "NTPC.NS", "JSWSTEEL.NS", "POWERGRID.NS", "BAJAJFINSV.NS", "GRASIM.NS",
    "ADANIENT.NS", "COALINDIA.NS", "TATASTEEL.NS", "DIVISLAB.NS", "HDFCLIFE.NS",
    "BAJAJ-AUTO.NS", "SBILIFE.NS", "DRREDDY.NS", "CIPLA.NS", "BPCL.NS", "INDUSINDBK.NS",
    "BRITANNIA.NS", "EICHERMOT.NS", "UPL.NS", "SHREECEM.NS", "HEROMOTOCO.NS",
    "M&M.NS", "HINDALCO.NS", "ONGC.NS", "APOLLOHOSP.NS", "TATACONSUM.NS"
]


In [47]:
results = []

for ticker in nifty_50_tickers:
    try:
        print(f"\n📊 Processing {ticker}...")
        df = fetch_and_prepare_data(ticker)
        X_train, X_test, y_train, y_test = prepare_lstm_data(df)
        model, loss, acc = train_lstm_model(X_train, X_test, y_train, y_test)

        # Save the model
        model.save(f"Models/{ticker.replace('.NS', '')}_lstm_model.h5")

        results.append({
            "Stock": ticker,
            "Accuracy": round(acc, 4),
            "Loss": round(loss, 4)
        })

    except Exception as e:
        print(f"❌ {ticker} failed: {e}")

# View summary of training results
pd.DataFrame(results)


[*********************100%***********************]  1 of 1 completed


📊 Processing RELIANCE.NS...



C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(row['RSI_14']) > 60) and (float(row['Close']) > float(row['SMA_50'])) and (float(row['MACD']) > float(row['MACD_Signal'])):
C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif (float(row['RSI_14']) < 40) and (float(row['Close']) < float(row['SMA_50'])) and (float(row['MACD']) < float(row['MACD_Signal'])):
c:\Users\heman\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  s


📊 Processing TCS.NS...


c:\Users\heman\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(row['RSI_14']) > 60) and (float(row['Close']) > float(row['SMA_50'])) and (float(row['MACD']) > float(row['MACD_Signal'])):


...


📊 Processing APOLLOHOSP.NS...


[*********************100%***********************]  1 of 1 completed
C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(row['RSI_14']) > 60) and (float(row['Close']) > float(row['SMA_50'])) and (float(row['MACD']) > float(row['MACD_Signal'])):
C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif (float(row['RSI_14']) < 40) and (float(row['Close']) < float(row['SMA_50'])) and (float(row['MACD']) < float(row['MACD_Signal'])):
c:\Users\heman\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using a


📊 Processing TATACONSUM.NS...


[*********************100%***********************]  1 of 1 completed
C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(row['RSI_14']) > 60) and (float(row['Close']) > float(row['SMA_50'])) and (float(row['MACD']) > float(row['MACD_Signal'])):
C:\Users\heman\AppData\Local\Temp\ipykernel_34088\2187632135.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  elif (float(row['RSI_14']) < 40) and (float(row['Close']) < float(row['SMA_50'])) and (float(row['MACD']) < float(row['MACD_Signal'])):
c:\Users\heman\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using a

,Stock,Accuracy,Loss
0,RELIANCE.NS,0.6429,0.6383
1,TCS.NS,0.7698,0.5520
2,INFY.NS,0.7063,0.6516
3,HDFCBANK.NS,0.6429,0.7298
4,ICICIBANK.NS,0.7302,0.5402
5,HINDUNILVR.NS,0.7302,0.5915
6,SBIN.NS,0.7778,0.5281
7,BHARTIARTL.NS,0.6349,0.7237
8,ITC.NS,0.7857,0.5156
9,LT.NS,0.6905,0.6496
